In [1]:
import pandas as pd

# Load the uploaded datasets
file_path_1 = './Dataset I.csv'
file_path_2 = './Dataset II.csv'

# Read the datasets
dataset_1 = pd.read_csv(file_path_1)
dataset_2 = pd.read_csv(file_path_2)

# Display the first few rows of each dataset to understand their structure
dataset_1_head = dataset_1.head()
dataset_2_head = dataset_2.head()

dataset_1_head, dataset_2_head


c:\Users\HP\.conda\envs\DM_ENV\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\HP\AppData\Local\Temp\ipykernel_5836\3902501075.py:8: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset_1 = pd.read_csv(file_path_1)


(                    Websites   Lable
 0                 google.com  Benign
 1             www.google.com  Benign
 2              microsoft.com  Benign
 3         data.microsoft.com  Benign
 4  events.data.microsoft.com  Benign,
                                                  URL    type
 0      https://www.unb.ca/cic/datasets/url-2016.html  Benign
 1    https://www.unb.ca/cic/datasets/andmal2020.html  Benign
 2        https://www.sec.cs.tu-bs.de/~danarp/drebin/  Benign
 3  https://github.com/traceflight/Android-Malware...  Benign
 4  https://ieee-dataport.org/documents/dataset-an...  Benign)

In [6]:
def extract_features(df, url_column):
    df['url_length'] = df[url_column].apply(len)
    df['num_special_chars'] = df[url_column].apply(lambda x: sum(1 for char in x if char in ['/', '?', '&', '=', '-', '_', '%', '.']))
    df['has_ip'] = df[url_column].apply(lambda x: any(part.isdigit() for part in x.split('.')))
    df['num_subdomains'] = df[url_column].apply(lambda x: len(x.split('.')) - 2 if 'http' not in x else len(x.split('/')[2].split('.')) - 2)
    df['top_level_domain'] = df[url_column].apply(lambda x: x.split('.')[-1].split('/')[0])
    return df



In [7]:
dataset_1_cleaned = dataset_1.dropna(subset=['Websites'])
dataset_2_cleaned = dataset_2.dropna(subset=['URL'])


In [8]:
def sanitize_and_clean_urls(df, url_column):
    # Ensure all entries are strings and non-empty
    df[url_column] = df[url_column].astype(str).str.strip()
    df = df[df[url_column].str.contains(r'\.', regex=True)]  # Keep valid URLs containing at least one '.'
    df = df[df[url_column].apply(lambda x: len(x.split('/')) > 2)]  # Ensure valid URL structure
    return df

# Re-clean datasets
dataset_1_cleaned = sanitize_and_clean_urls(dataset_1_cleaned, 'Websites')
dataset_2_cleaned = sanitize_and_clean_urls(dataset_2_cleaned, 'URL')

dataset_1_features = extract_features(dataset_1_cleaned, 'Websites')
dataset_2_features = extract_features(dataset_2_cleaned, 'URL')

C:\Users\HP\AppData\Local\Temp\ipykernel_26476\591616003.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[url_column] = df[url_column].astype(str).str.strip()


In [11]:
# Display the first few rows of each dataset directly
dataset_1_features.head()


,Websites,Lable,url_length,num_special_chars,has_ip,num_subdomains,top_level_domain
4002,http://www.marketingbyinternet.com/mo/e56508df...,Malicious,71,7,False,1,com
4003,http://www.824555.com/app/member/SportOption.p...,Malicious,67,12,True,1,php?uid=guest&langx=gb
4004,https://docs.google.com/spreadsheet/viewform?f...,Malicious,87,8,False,1,com
4006,http://www.martin-busker.de/administrator/help...,Malicious,153,16,False,1,php
4009,http://www.bimabn.com/1-configurazione-support...,Malicious,113,15,False,1,store-contatta


In [12]:
# Display the first few rows of each dataset directly
dataset_2_features.head()


,URL,type,url_length,num_special_chars,has_ip,num_subdomains,top_level_domain
0,https://www.unb.ca/cic/datasets/url-2016.html,Benign,45,9,False,1,html
1,https://www.unb.ca/cic/datasets/andmal2020.html,Benign,47,8,False,1,html
2,https://www.sec.cs.tu-bs.de/~danarp/drebin/,Benign,43,10,False,3,de
3,https://github.com/traceflight/Android-Malware...,Benign,55,7,False,0,com
4,https://ieee-dataport.org/documents/dataset-an...,Benign,75,9,False,0,org


In [13]:
dataset_2_features.shape

(4999, 7)

In [14]:
dataset_1_features.shape

(957, 7)

In [15]:
combined_dataset = pd.concat([
    dataset_1_features.rename(columns={"Websites": "URL", "Lable": "Type"}),
    dataset_2_features.rename(columns={"type": "Type"})
], ignore_index=True)


In [16]:
combined_dataset.shape

(5956, 7)

In [18]:
combined_dataset.to_csv('combined_dataset.csv', index=False)

In [19]:
combined_dataset.head()

,URL,Type,url_length,num_special_chars,has_ip,num_subdomains,top_level_domain
0,http://www.marketingbyinternet.com/mo/e56508df...,Malicious,71,7,False,1,com
1,http://www.824555.com/app/member/SportOption.p...,Malicious,67,12,True,1,php?uid=guest&langx=gb
2,https://docs.google.com/spreadsheet/viewform?f...,Malicious,87,8,False,1,com
3,http://www.martin-busker.de/administrator/help...,Malicious,153,16,False,1,php
4,http://www.bimabn.com/1-configurazione-support...,Malicious,113,15,False,1,store-contatta


In [20]:
combined_dataset.to_csv('combined_dataset.csv')

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
# Create separate label encoders for 'Type' and 'top_level_domain'
type_encoder = LabelEncoder()
top_level_domain_encoder = LabelEncoder()

# Encode the 'Type' column as the target variable
combined_dataset['Type_encoded'] = type_encoder.fit_transform(combined_dataset['Type'])

# Encode 'top_level_domain' as a feature
combined_dataset['top_level_domain_encoded'] = top_level_domain_encoder.fit_transform(combined_dataset['top_level_domain'])

# Select features and target variable
features = ['url_length', 'num_special_chars', 'has_ip', 'num_subdomains', 'top_level_domain_encoded']
X = combined_dataset[features]
y = combined_dataset['Type_encoded']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
classification_report_output = classification_report(y_test, y_pred, target_names=type_encoder.classes_)

classification_report_output


'              precision    recall  f1-score   support\n\n      Benign       0.97      0.95      0.96       481\n   Malicious       0.97      0.98      0.97       711\n\n    accuracy                           0.97      1192\n   macro avg       0.97      0.97      0.97      1192\nweighted avg       0.97      0.97      0.97      1192\n'

In [ ]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Make predictions  
xgb_y_pred = xgb_model.predict(X_test)

# Evaluate the model
xgb_classification_report = classification_report(y_test, xgb_y_pred, target_names=type_encoder.classes_)

xgb_classification_report

'              precision    recall  f1-score   support\n\n      Benign       0.97      0.95      0.96       481\n   Malicious       0.97      0.98      0.97       711\n\n    accuracy                           0.97      1192\n   macro avg       0.97      0.97      0.97      1192\nweighted avg       0.97      0.97      0.97      1192\n'

In [24]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for optimization
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform grid search with cross-validation to find the best parameters
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring='precision_weighted',  # Optimize for higher precision
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1
)

# Fit the grid search to the training data
grid_search.fit(X_train, y_train)

# Get the best model and parameters
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

# Evaluate the optimized model
optimized_y_pred = best_model.predict(X_test)
optimized_classification_report = classification_report(y_test, optimized_y_pred, target_names=type_encoder.classes_)

best_params, optimized_classification_report

Fitting 3 folds for each of 108 candidates, totalling 324 fits


({'max_depth': 20,
  'min_samples_leaf': 1,
  'min_samples_split': 5,
  'n_estimators': 200},
 '              precision    recall  f1-score   support\n\n      Benign       0.97      0.95      0.96       481\n   Malicious       0.97      0.98      0.97       711\n\n    accuracy                           0.97      1192\n   macro avg       0.97      0.97      0.97      1192\nweighted avg       0.97      0.97      0.97      1192\n')

# dynammic qr codes  

In [27]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Re-load the original combined dataset
file_path_combined = './combined_dataset.csv'
combined_dataset = pd.read_csv(file_path_combined)

# Create a copy for the new dynamic dataset
dynamic_dataset = combined_dataset.copy()

# Add 'created_at', 'expires_at', and 'transaction_at' columns to dynamic_dataset
np.random.seed(42)  # For reproducibility
current_time = datetime.now()

# Generate 'created_at' within the last 30 days
dynamic_dataset['created_at'] = [
    current_time - timedelta(days=np.random.randint(0, 30), hours=np.random.randint(0, 24))
    for _ in range(len(dynamic_dataset))
]

# Generate 'expires_at' by adding a random validity period (1 to 5 hours) to 'created_at'
dynamic_dataset['expires_at'] = dynamic_dataset['created_at'] + pd.to_timedelta(
    np.random.randint(1, 6, size=len(dynamic_dataset)), unit='h'
)

# Generate 'transaction_at' as random timestamps within or outside the valid range
dynamic_dataset['transaction_at'] = [
    created + pd.to_timedelta(np.random.randint(-3, 8), unit='h')  # Random transaction time
    for created in dynamic_dataset['created_at']
]

# Update 'Type' based on the condition
# If the transaction_at doesn't fall in the valid range and the label is benign, mark as malicious
dynamic_dataset['Type'] = dynamic_dataset.apply(
    lambda row: 'Malicious' if (
        row['Type'] == 'Benign' and not (row['created_at'] <= row['transaction_at'] <= row['expires_at'])
    ) else row['Type'], axis=1
)
dynamic_dataset

,Unnamed: 0,URL,Type,url_length,num_special_chars,has_ip,num_subdomains,top_level_domain,created_at,expires_at,transaction_at
0,0,http://www.marketingbyinternet.com/mo/e56508df...,Malicious,71,7,False,1,com,2024-12-13 05:29:26.596129,2024-12-13 08:29:26.596129,2024-12-13 11:29:26.596129
1,1,http://www.824555.com/app/member/SportOption.p...,Malicious,67,12,True,1,php?uid=guest&langx=gb,2024-11-21 10:29:26.596129,2024-11-21 12:29:26.596129,2024-11-21 13:29:26.596129
2,2,https://docs.google.com/spreadsheet/viewform?f...,Malicious,87,8,False,1,com,2024-12-09 17:29:26.596129,2024-12-09 19:29:26.596129,2024-12-09 16:29:26.596129
3,3,http://www.martin-busker.de/administrator/help...,Malicious,153,16,False,1,php,2024-11-21 04:29:26.596129,2024-11-21 05:29:26.596129,2024-11-21 03:29:26.596129
4,4,http://www.bimabn.com/1-configurazione-support...,Malicious,113,15,False,1,store-contatta,2024-12-13 06:29:26.596129,2024-12-13 07:29:26.596129,2024-12-13 05:29:26.596129
...,...,...,...,...,...,...,...,...,...,...,...
5951,5951,http://www.driduct.com/home,Malicious,27,5,False,1,com,2024-12-16 10:29:26.596129,2024-12-16 12:29:26.596129,2024-12-16 16:29:26.596129
5952,5952,http://greg-miller.com/wp-content/themes/twent...,Malicious,249,19,False,0,php?cmd=login_submit&amp;id=8c4aa22d352c4d9239...,2024-12-11 18:29:26.596129,2024-12-11 23:29:26.596129,2024-12-11 17:29:26.596129
5953,5953,http://greg-miller.com/wp-content/themes/twent...,Malicious,249,19,False,0,php?cmd=login_submit&amp;id=77700b6afcca759deb...,2024-12-10 02:29:26.596129,2024-12-10 07:29:26.596129,2024-12-10 08:29:26.596129
5954,5954,http://greg-miller.com/wp-content/themes/twent...,Malicious,249,19,False,0,php?cmd=login_submit&amp;id=291e7d25cb52f8e2e2...,2024-12-07 17:29:26.596129,2024-12-07 22:29:26.596129,2024-12-07 16:29:26.596129


In [28]:
combined_dataset['Type'].value_counts()

Type
Malicious    3456
Benign       2500
Name: count, dtype: int64

In [29]:
dynamic_dataset['Type'].value_counts()

Type
Malicious    5074
Benign        882
Name: count, dtype: int64

In [30]:
# Identify transactions that were altered from benign to malicious
altered_to_malicious = dynamic_dataset[(dynamic_dataset['Type'] == 'Malicious') & (combined_dataset['Type'] == 'Benign')]

# Adjust the timestamps for these transactions to make them valid (i.e., ensure transaction_at falls in the valid period)
healthy_versions = altered_to_malicious.copy()
healthy_versions['transaction_at'] = healthy_versions['created_at'] + timedelta(minutes=1)  # Make transaction_at just after created_at
healthy_versions['Type'] = 'Benign'  # Reset the label to benign

# Append the healthy versions back to the dynamic dataset
dynamic_dataset_updated = pd.concat([dynamic_dataset, healthy_versions], ignore_index=True)


In [31]:
dynamic_dataset_updated['Type'].value_counts()

Type
Malicious    5074
Benign       2500
Name: count, dtype: int64

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

# Preprocessing
# Encode categorical variables
type_encoder = LabelEncoder()
top_level_domain_encoder = LabelEncoder()

# Encode the 'Type' column as the target variable
dynamic_dataset_updated['Type_encoded'] = type_encoder.fit_transform(dynamic_dataset_updated['Type'])

# Encode 'top_level_domain' as a feature
dynamic_dataset_updated['top_level_domain_encoded'] = top_level_domain_encoder.fit_transform(dynamic_dataset_updated['top_level_domain'])

# Select features and target variable
features = ['url_length', 'num_special_chars', 'has_ip', 'num_subdomains', 'top_level_domain_encoded']
# Add new timestamp-based features for the model
dynamic_dataset_updated['transaction_within_valid_period'] = (
    (dynamic_dataset_updated['created_at'] <= dynamic_dataset_updated['transaction_at']) &
    (dynamic_dataset_updated['transaction_at'] <= dynamic_dataset_updated['expires_at'])
).astype(int)

features += ['transaction_within_valid_period']
X = dynamic_dataset_updated[features]
y = dynamic_dataset_updated['Type_encoded']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
y_pred = rf_model.predict(X_test)

# Evaluate the model
classification_report_output = classification_report(y_test, y_pred, target_names=type_encoder.classes_)

classification_report_output


'              precision    recall  f1-score   support\n\n      Benign       0.99      0.98      0.99       505\n   Malicious       0.99      1.00      0.99      1010\n\n    accuracy                           0.99      1515\n   macro avg       0.99      0.99      0.99      1515\nweighted avg       0.99      0.99      0.99      1515\n'

In [36]:
X

,url_length,num_special_chars,has_ip,num_subdomains,top_level_domain_encoded,transaction_within_valid_period
0,71,7,False,1,130,0
1,67,12,True,1,555,0
2,87,8,False,1,130,0
3,153,16,False,1,366,0
4,113,15,False,1,589,0
...,...,...,...,...,...,...
7569,25,4,False,1,130,1
7570,22,4,False,1,130,1
7571,21,4,False,1,130,1
7572,21,4,False,1,332,1


In [33]:
dynamic_dataset_updated.head()

,Unnamed: 0,URL,Type,url_length,num_special_chars,has_ip,num_subdomains,top_level_domain,created_at,expires_at,transaction_at,Type_encoded,transaction_within_valid_period
0,0,http://www.marketingbyinternet.com/mo/e56508df...,Malicious,71,7,False,1,com,2024-12-13 05:29:26.596129,2024-12-13 08:29:26.596129,2024-12-13 11:29:26.596129,1,0
1,1,http://www.824555.com/app/member/SportOption.p...,Malicious,67,12,True,1,php?uid=guest&langx=gb,2024-11-21 10:29:26.596129,2024-11-21 12:29:26.596129,2024-11-21 13:29:26.596129,1,0
2,2,https://docs.google.com/spreadsheet/viewform?f...,Malicious,87,8,False,1,com,2024-12-09 17:29:26.596129,2024-12-09 19:29:26.596129,2024-12-09 16:29:26.596129,1,0
3,3,http://www.martin-busker.de/administrator/help...,Malicious,153,16,False,1,php,2024-11-21 04:29:26.596129,2024-11-21 05:29:26.596129,2024-11-21 03:29:26.596129,1,0
4,4,http://www.bimabn.com/1-configurazione-support...,Malicious,113,15,False,1,store-contatta,2024-12-13 06:29:26.596129,2024-12-13 07:29:26.596129,2024-12-13 05:29:26.596129,1,0
